In [2]:
import pandas as pd
df_multiple_notes = pd.read_csv('/Users/lifuchen/Desktop/OpenNotes/data/multiple_note.csv')

In [3]:
df_multiple_notes.head()

,subject_id,hadm_id,seq_num,icd_code,icd_version,diagnosis_category,gender,anchor_age,anchor_year,anchor_year_group,...,days_since_anchor,cleaned_note_text,text_kept_exact_sentence_match,text_kept_exact_ngram_match,text_kept_levenshtein_match,similarity_scores_levenshtein_match,text_kept_sentence_transformer_match,similarity_scores_sentence_transformer_match,text_kept_lcs_match,similarity_scores_lcs_match
0,10070024,26769931,1,F323,10,Major Depression,F,23,2142,2014 - 2016,...,1124,\nName: ___ Unit No: ___...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,10070024,26398294,1,F3189,10,Bipolar Disorder,F,23,2142,2014 - 2016,...,1135,"Chief Complaint:\n""I made a mistake. ""\n \nMaj...","['chief complaint: ""i made a mistake.', '"" maj...","['no: ___', 'chief complaint: ""i made a mistak...","['chief complaint: ""i made a mistake.', '"" maj...","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.5, 0.54154727...","['chief complaint: ""i made a mistake.', '"" maj...","[1.0, 1.0000001192092896, 1.0000001192092896, ...","['chief complaint: ""i made a mistake.', '"" maj...","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.5, 0.37735849..."
2,10080985,24679803,1,F332,10,Major Depression,F,22,2179,2014 - 2016,...,132,\nName: ___ Unit No: ___...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,10080985,26523165,1,F329,10,Major Depression,F,22,2179,2014 - 2016,...,130,Chief Complaint:\ngenetic predisposition to br...,['sex: f service: emergency allergies: amoxici...,"['no: ___', 'patient herself denies any signif...",['chief complaint: genetic predisposition to b...,"[1.0, 1.0, 1.0, 1.0, 1.0, 0.8837209302325582, ...",['chief complaint: genetic predisposition to b...,"[1.0000001192092896, 1.0, 1.0, 1.0, 1.0, 0.932...",['chief complaint: genetic predisposition to b...,"[1.0, 1.0, 1.0, 1.0, 1.0, 0.875, 0.30555555555..."
4,10266157,29245849,1,F332,10,Major Depression,F,76,2194,2011 - 2013,...,1755,\nName: ___ Unit No: ___\...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
import spacy

# Load English tokenizer, tagger, parser and NER
nlp = spacy.load("en_core_web_sm")

def split_into_sentences(text):
    doc = nlp(text)
    return [
        " ".join(sent.text.lower().split())  # lowercase + strip extra internal spaces
        for sent in doc.sents
        if sent.text.strip()
    ]

In [15]:
def highlight_removed_sentences_colored(original_text, kept_sentences_by_method):
    original_sentences = split_into_sentences(original_text)

    # Track each sentence instance separately
    sentence_status = {i: {"sent": sent, "removed_by": []} for i, sent in enumerate(original_sentences)}

    for method, kept in kept_sentences_by_method.items():
        kept_set = set(kept)
        for i, sent in enumerate(original_sentences):
            if sent not in kept_set:
                sentence_status[i]["removed_by"].append(method)

    # Generate HTML
    html_output = []
    for i in range(len(original_sentences)):
        sent = sentence_status[i]["sent"]
        removed_by = sentence_status[i]["removed_by"]

        if not removed_by:
            html_output.append(f"<p>{sent}</p>")
        else:
            color = method_colors.get(removed_by[0], "gray")
            method_list = ", ".join(sorted(set(removed_by)))
            html_output.append(
                f"<p><span style='color:{color}; text-decoration:line-through;' title='Removed by {method_list}'>"
                f"{sent}</span> <em style='color:{color}'>(removed by {method_list})</em></p>"
            )

    return "\n".join(html_output)


In [6]:
import ast

# Attempt to parse the kept_sentences if they are stringified
def parse_kept_sentences(x):
    if isinstance(x, list):
        return x
    elif isinstance(x, str): # actually the case
        try:
            return ast.literal_eval(x)
        except:
            return []
    return []


In [17]:
methods_to_check = {
    "Exact Match": "text_kept_exact_sentence_match",
    "Ngram": "text_kept_exact_ngram_match",
    "Levenshtein": "text_kept_levenshtein_match",
    "Sentence Transformer": "text_kept_sentence_transformer_match",
    "LCS": "text_kept_lcs_match"
}

method_colors = {
    "Exact Sentence Match": "grey",
    "Ngram": "orange",
    "Levenshtein": "purple",
    "Sentence Transformer": "blue",
    "LCS": "green"
}

row_idx = 3  # change to any row you want to inspect
row = df_multiple_notes.iloc[row_idx]
original_text = row["text"]

# Gather kept sentences for each method
kept_sentences_by_method = {
    method: parse_kept_sentences(row.get(col))
    for method, col in methods_to_check.items()
}

# Generate and save HTML
combined_html = f"""
<!DOCTYPE html>
<html>
<head>
    <meta charset="utf-8">
    <title>Deduplication Comparison</title>
</head>
<body>
    <h1>Combined Highlighting of Removed Sentences (Color-Coded by Method)</h1>
    <ul>
        {''.join([f"<li style='color:{color}'>{method}</li>" for method, color in method_colors.items()])}
    </ul>
    {highlight_removed_sentences_colored(original_text, kept_sentences_by_method)}
</body>
</html>
"""

with open("/Users/lifuchen/Desktop/deduplication_colored_2.html", "w", encoding="utf-8") as f:
    f.write(combined_html)

print("✅ Saved to deduplication_colored_2.html")


✅ Saved to deduplication_colored_2.html


In [8]:
sentence = "3. substance use disorders: #)"
tokens = [token.text for token in nlp(sentence)]
print(tokens)


['3', '.', 'substance', 'use', 'disorders', ':', '#', ')']


In [16]:
methods_to_check = {
    "Exact Match": "text_kept_exact_sentence_match",
    #"Ngram": "text_kept_exact_ngram_match",
    "Levenshtein": "text_kept_levenshtein_match",
    "Sentence Transformer": "text_kept_sentence_transformer_match",
    "LCS": "text_kept_lcs_match"
}

method_colors = {
    "Exact Sentence Match": "grey",
    #"Ngram": "orange",
    "Levenshtein": "purple",
    "Sentence Transformer": "blue",
    "LCS": "green"
}

row_idx = 3  # change to any row you want to inspect
row = df_multiple_notes.iloc[row_idx]
original_text = row["text"]

# Gather kept sentences for each method
kept_sentences_by_method = {
    method: parse_kept_sentences(row.get(col))
    for method, col in methods_to_check.items()
}

# Generate and save HTML
combined_html = f"""
<!DOCTYPE html>
<html>
<head>
    <meta charset="utf-8">
    <title>Deduplication Comparison</title>
</head>
<body>
    <h1>Combined Highlighting of Removed Sentences (Color-Coded by Method)</h1>
    <ul>
        {''.join([f"<li style='color:{color}'>{method}</li>" for method, color in method_colors.items()])}
    </ul>
    {highlight_removed_sentences_colored(original_text, kept_sentences_by_method)}
</body>
</html>
"""

with open("/Users/lifuchen/Desktop/deduplication_colored_no_ngram_2.html", "w", encoding="utf-8") as f:
    f.write(combined_html)

print("✅ Saved to deduplication_colored_no_ngram_2.html")


✅ Saved to deduplication_colored_no_ngram_2.html
